<a href="https://colab.research.google.com/github/ShreyasSam2004/AI_flight_agent/blob/main/AI_flight_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio openai python-dotenv pillow
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxxxxxxxxxxxxxx"



In [ ]:
print(repr(os.getenv("OPENAI_API_KEY")))



'sk-proj-xxxxxxxxxxxxxxxx'


In [ ]:
from google.colab import userdata
from openai import OpenAI

# Load key from Colab secrets
api_key = userdata.get("OPENAI_API_KEY")

# Initialize client
client = OpenAI(api_key=api_key)

# Quick test
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role":"user","content":"hello"}]
)
print(resp.choices[0].message.content)


Hello! How can I assist you today?


In [ ]:
# --- Install dependencies ---
!pip install openai gradio pillow

# --- Imports ---
import os, json, base64, traceback
import gradio as gr
from io import BytesIO
from PIL import Image
from google.colab import userdata
from openai import OpenAI

# --- Setup API key ---
try:
    api_key = userdata.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    print("✅ OpenAI client initialized successfully")
except Exception as e:
    print(f"❌ Error setting up OpenAI client: {e}")
    client = None

# --- System Message ---
system_message = (
    "You are FlightAI, a helpful assistant for an airline. "
    "Give short, courteous answers. "
    "Always include flight costs, timings, and airline names if available. "
    "If you don't know, say so."
)

# --- Mock Flight Database ---
flight_data = {
    ("india", "usa"): [
        {"airline": "Air India", "price": "$750", "timing": "10:00 AM - 6:00 PM"},
        {"airline": "United Airlines", "price": "$820", "timing": "11:30 AM - 8:00 PM"},
    ],
    ("new york", "london"): [
        {"airline": "British Airways", "price": "$799", "timing": "7:00 PM - 7:00 AM"},
        {"airline": "Delta", "price": "$850", "timing": "8:00 PM - 8:30 AM"},
    ],
    ("london", "paris"): [
        {"airline": "Air France", "price": "$120", "timing": "9:00 AM - 11:00 AM"},
        {"airline": "British Airways", "price": "$150", "timing": "1:00 PM - 3:00 PM"},
    ],
    ("tokyo", "singapore"): [
        {"airline": "Singapore Airlines", "price": "$450", "timing": "2:00 PM - 8:00 PM"},
        {"airline": "ANA", "price": "$500", "timing": "5:00 PM - 11:00 PM"},
    ],
    ("dubai", "sydney"): [
        {"airline": "Emirates", "price": "$1200", "timing": "8:00 AM - 11:00 PM"},
        {"airline": "Qantas", "price": "$1300", "timing": "9:30 AM - 12:30 AM"},
    ],
    ("berlin", "rome"): [
        {"airline": "Lufthansa", "price": "$200", "timing": "6:00 AM - 8:00 AM"},
        {"airline": "ITA Airways", "price": "$180", "timing": "12:00 PM - 2:00 PM"},
    ],
    ("mumbai", "dubai"): [
        {"airline": "Emirates", "price": "$300", "timing": "9:00 AM - 11:00 AM"},
        {"airline": "Indigo", "price": "$250", "timing": "4:00 PM - 6:00 PM"},
    ],
    ("toronto", "vancouver"): [
        {"airline": "Air Canada", "price": "$220", "timing": "8:00 AM - 11:00 AM"},
        {"airline": "WestJet", "price": "$200", "timing": "3:00 PM - 6:00 PM"},
    ],
    ("los angeles", "tokyo"): [
        {"airline": "ANA", "price": "$950", "timing": "1:00 PM - 7:00 PM"},
        {"airline": "Japan Airlines", "price": "$990", "timing": "11:00 AM - 5:30 PM"},
    ],
    ("chicago", "mexico city"): [
        {"airline": "Aeromexico", "price": "$400", "timing": "7:00 AM - 11:00 AM"},
        {"airline": "United Airlines", "price": "$420", "timing": "5:00 PM - 9:00 PM"},
    ],
    ("hong kong", "bangkok"): [
        {"airline": "Thai Airways", "price": "$280", "timing": "10:00 AM - 12:00 PM"},
        {"airline": "Cathay Pacific", "price": "$300", "timing": "4:00 PM - 6:00 PM"},
    ],
    ("sao paulo", "madrid"): [
        {"airline": "Iberia", "price": "$780", "timing": "2:00 PM - 6:00 AM"},
        {"airline": "LATAM", "price": "$820", "timing": "9:00 PM - 1:00 PM"},
    ],
    ("london", "dubai"): [
        {"airline": "Emirates", "price": "$600", "timing": "8:00 AM - 6:00 PM"},
        {"airline": "British Airways", "price": "$650", "timing": "9:30 AM - 7:00 PM"},
    ],
    ("paris", "new york"): [
        {"airline": "Air France", "price": "$720", "timing": "10:00 AM - 1:00 PM"},
        {"airline": "Delta", "price": "$750", "timing": "3:00 PM - 6:00 PM"},
    ],
}

# --- Flight Utility ---
def get_flight_details(origin, destination):
    try:
        key = (origin.lower().strip(), destination.lower().strip())
        flights = flight_data.get(key)
        if not flights:
            return "❌ Sorry, I don't have flight info for that route right now."
        response = f"Here are some flights from {origin.title()} to {destination.title()}:\n\n"
        for f in flights:
            response += f"- ✈️ {f['airline']}: {f['price']}, {f['timing']}\n"
        return response
    except Exception as e:
        print(f"Error in get_flight_details: {e}")
        return "❌ Sorry, there was an error retrieving flight information."

# --- Tool schema ---
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_details",
            "description": "Get available flights (airline, price, timings) between two cities.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_city": {"type": "string", "description": "The departure city"},
                    "destination_city": {"type": "string", "description": "The destination city"},
                },
                "required": ["origin_city", "destination_city"],
            },
        },
    }
]

def handle_tool_call(message):
    try:
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        origin = arguments.get("origin_city")
        destination = arguments.get("destination_city")
        details = get_flight_details(origin, destination)
        response = {
            "role": "tool",
            "content": details,
            "tool_call_id": tool_call.id,
        }
        return response, origin, destination
    except Exception as e:
        print(f"Error in handle_tool_call: {e}")
        error_response = {
            "role": "tool",
            "content": "❌ Sorry, there was an error processing your flight request.",
            "tool_call_id": message.tool_calls[0].id,
        }
        return error_response, None, None

# --- Image Generator ---
def artist(city):
    try:
        if not client:
            print("OpenAI client not available for image generation")
            return None

        # Use DALL-E 3 instead of gpt-image-1 (which doesn't exist)
        image_response = client.images.generate(
            model="dall-e-3",
            prompt=f"A stylish travel poster of {city}, with landmarks, planes, and cultural highlights, in vibrant pop-art style",
            size="1024x1024",  # DALL-E 3 supports different sizes
            n=1,
            response_format="b64_json",
        )
        image_base64 = image_response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        return Image.open(BytesIO(image_data))
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

# --- Chat Logic ---
MODEL = "gpt-4o-mini"

def chat(history):
    try:
        if not client:
            error_msg = "❌ OpenAI client is not available. Please check your API key."
            history.append({"role": "assistant", "content": error_msg})
            return history, None

        messages = [{"role": "system", "content": system_message}] + history
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        image = None

        if response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_response, origin, destination = handle_tool_call(message)

            # Convert message to dict format for consistency
            messages.append({
                "role": "assistant",
                "content": message.content,
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": tc.type,
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    } for tc in message.tool_calls
                ]
            })
            messages.append(tool_response)

            # Generate image if destination is available
            if destination:
                image = artist(destination)

            response = client.chat.completions.create(model=MODEL, messages=messages)

        reply = response.choices[0].message.content
        history.append({"role": "assistant", "content": reply})
        return history, image

    except Exception as e:
        print(f"Error in chat function: {e}")
        traceback.print_exc()
        error_msg = f"❌ Sorry, I encountered an error: {str(e)}"
        history.append({"role": "assistant", "content": error_msg})
        return history, None

# --- Gradio UI ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="slate", secondary_hue="slate").set(
    body_background_fill="#111827",
    background_fill_primary="#1f2937",
    background_fill_secondary="#374151",
    block_background_fill="#1f2937",
    block_border_color="#4b5563",
    input_background_fill="#374151",
    button_primary_background_fill="#4b5563",
    button_primary_background_fill_hover="#6b7280",
    button_secondary_background_fill="#374151"
), css="""
.gradio-container {background: #111827 !important;}
.dark {background: #111827 !important;}
body {background: #111827 !important;}
.message.bot, .message.assistant {
    color: #ffffff !important;
}
.chatbot .message-wrap .message {
    color: #ffffff !important;
}
.chatbot .message.bot .message-content {
    color: #ffffff !important;
}
.chatbot .message.assistant .message-content {
    color: #ffffff !important;
}
.chatbot div[data-testid="bot"] {
    color: #ffffff !important;
}
.chatbot .bot {
    color: #ffffff !important;
}
.chatbot .message {
    color: #ffffff !important;
}
.message-content {
    color: #ffffff !important;
}
""") as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, type="pil")
    with gr.Row():
        entry = gr.Textbox(label="Chat with FlightAI:", placeholder="e.g., 'I need flights from Tokyo to Singapore'")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        if not message.strip():
            return "", history
        history.append({"role": "user", "content": message})
        return "", history

    def clear_chat():
        return None, None

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(clear_chat, inputs=None, outputs=[chatbot, image_output], queue=False)

# Launch the app
if __name__ == "__main__":
    ui.launch(inbrowser=True, debug=True)


✅ OpenAI client initialized successfully
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f2d1804d592c4c5dc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
